In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import nmslib
from lang_model_utils import load_lm_vocab, Query2Emb
from general_utils import create_nmslib_search_index

input_path = Path('./data/processed_data/')
code2emb_path = Path('./data/code2emb/')
output_path = Path('./data/search')
output_path.mkdir(exist_ok=True)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests im

In [5]:
# load test data

test_tokens = np.load('./data/seq2seq/test.tokens.npy')
test_apiseq = np.load('./data/seq2seq/test.apiseq.npy')
test_methname = np.load('./data/seq2seq/test.methname.npy')
test_description = np.load('./data/seq2seq/test.desc.npy')

In [6]:
test_tokens.shape

(179248, 55)

In [7]:
test_description.shape

(179248, 15)

In [11]:
test_description = np.load('./data/lang_model_emb/avg_emb_dim500_test_v2.npy')

In [12]:
test_description.shape

(179248, 500)

In [13]:
rng = np.random.RandomState(42)
validation_indices = np.random.choice(test_description.shape[0], replace=False, size=10000)

In [18]:
from keras.models import load_model
from pathlib import Path
import numpy as np
code2emb_path = Path('./data/code2emb/')
code2emb_model = load_model(code2emb_path/'code2emb_model_concatenate_.hdf5')

In [24]:
eval_code_vecs = code2emb_model.predict([test_tokens, test_apiseq, test_methname], batch_size=2048)

In [30]:
search_index = create_nmslib_search_index(eval_code_vecs[validation_indices])

In [31]:
eval_doc_vecs = test_description[validation_indices]

In [32]:
import math
def ACC(real, predict):
        sum = 0.0
        for val in real:
            try:
                index = predict.index(val)
            except ValueError:
                index = -1
            if index != -1:
                sum = sum + 1
        return sum / float(len(real))

def MAP(real, predict):
    sum = 0.0
    for id, val in enumerate(real):
        try:
            index = predict.index(val)
        except ValueError:
            index = -1
        if index != -1:
            sum = sum + (id + 1) / float(index + 1)
    return sum / float(len(real))

def MRR(real, predict):
    sum = 0.0
    for val in real:
        try:
            index = predict.index(val)
        except ValueError:
            index = -1
        if index != -1:
            sum = sum + 1.0 / float(index + 1)
    return sum / float(len(real))

def NDCG(real, predict):
    dcg = 0.0
    idcg = IDCG(len(real))
    for i, predictItem in enumerate(predict):
        if predictItem in real:
            itemRelevance = 1
            rank = i + 1
            dcg += (math.pow(2, itemRelevance) - 1.0) * (math.log(2) / math.log(rank + 1))
    return dcg / float(idcg)

def IDCG(n):
    idcg = 0
    itemRelevance = 1
    for i in range(n):
        idcg += (math.pow(2, itemRelevance) - 1.0) * (math.log(2) / math.log(i + 2))
    return idcg

In [33]:
import torch.nn.functional as F

accs, mrrs, maps, ndcgs = [], [], [], []
for i in range(eval_doc_vecs.shape[0]):
    doc_vec = eval_doc_vecs[i]
    predict, dists = search_index.knnQuery(doc_vec, k=10)
    real = [i]
    predict = [int(k) for k in predict]
    accs.append(ACC(real, predict))
    mrrs.append(MRR(real, predict))
    maps.append(MAP(real, predict))
    ndcgs.append(NDCG(real, predict))
print('ACC={}, MRR={}, MAP={}, nDCG={}'.format(np.mean(accs), np.mean(mrrs), np.mean(maps),
                                                     np.mean(ndcgs)))

ACC=0.4249, MRR=0.25163769841269834, MAP=0.25163769841269834, nDCG=0.2927080688249676
